# Лаб-3. Рекомендательные системы

In [4]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import numpy as np

import pandas as pd

In [5]:
# Выбираем девайс
device = "cpu" if torch.cuda.is_available() else "cpu"
print(f'Device: {device}')

Device: cpu


В качестве датасета будем использовать MovieLens

https://grouplens.org/datasets/movielens/

А именно, самый маленький вариант со 100 тыс. оценок

https://files.grouplens.org/datasets/movielens/ml-latest-small.zip


In [6]:
# Для загрузки датасета напишем свою реализацию класса Dataset
class MovielensDataset(Dataset):
    r"""seed должен быть одинаковым для обучающей и тренировочной выборки"""
    def __init__(self, source, train=True, seed=1):
        ratings      = pd.read_csv(rf"{source}\ratings.csv")
        self.movies  = pd.read_csv(rf"{source}\movies.csv")

        # Преобразовываем Id фильмов в индексы в таблице movies
        x = self.movies.loc[:,['movieId']]
        x['movieId'], x.index = x.index, x['movieId'].values
        ratings['movieId'] = ratings['movieId'].map(x.to_dict()['movieId'])

        # делим датасет 80% на 20%
        train_data = ratings.sample(frac=0.8, random_state=seed)
        test_data  = ratings.drop(train_data.index)

        self.ratings = train_data if train else test_data

    def __len__(self):
        return len(self.ratings)

    def __getitem__(self, idx):
        sample = self.ratings.iloc[idx]
        return {
            "user": torch.LongTensor([sample['userId']]),
            "movie": torch.LongTensor([sample['movieId']]),
            "rating": torch.FloatTensor([sample['rating']])
        }
    def add_new_user(self, seed=1, num_movies=20):
        """Метод для добавления нового пользователя с его оценками"""
        # Уникальный идентификатор для нового пользователя
        new_user_id = self.ratings['userId'].max() + 1
    
        # Выбор случайных фильмов и присвоение им случайных рейтингов от 1.0 до 5.0
        np.random.seed(seed)
        sampled_movie_ids = self.ratings['movieId'].unique()[:20]
        sampled_ratings = np.round(np.random.uniform(1.0, 5.0, num_movies) * 2) / 2
    
        #Создание данных для нового пользователя
        new_user_ratings = pd.DataFrame({
            'userId': new_user_id,
            'movieId': sampled_movie_ids,
            'rating': sampled_ratings,
            'timestamp': int(pd.Timestamp.now().timestamp())
        })
    
        # Добавление нового пользователя в рейтинг
        self.ratings = pd.concat([self.ratings, new_user_ratings], ignore_index=False)
    
        # Возвращаем новый идентификатор пользователя
        return new_user_id


batch_size = 200

sataset_source = r'.\ml-latest-small'

movielens_train = MovielensDataset(sataset_source, train=True)
movielens_test  = MovielensDataset(sataset_source, train=False)

print("До добавления нового пользователя:")
print(movielens_train.ratings['userId'].min(), movielens_train.ratings['userId'].max())
print(movielens_train.ratings['movieId'].min(), movielens_train.ratings['movieId'].max())
print(movielens_train.ratings.sort_values(by='userId').reset_index(drop=True))

# Добавление нового пользователя
new_user_id = movielens_train.add_new_user()

print("После добавления нового пользователя:")
print(movielens_train.ratings['userId'].min(), movielens_train.ratings['userId'].max())
print(movielens_train.ratings['movieId'].min(), movielens_train.ratings['movieId'].max())
print(movielens_train.ratings.sort_values(by='userId').reset_index(drop=True))

# Проверим, добавился ли новый пользователь
new_user_id = movielens_train.ratings['userId'].max()  # Это должен быть ID нового пользователя

# Проверим, есть ли оценки для нового пользователя
new_user_ratings = movielens_train.ratings[movielens_train.ratings['userId'] == new_user_id]

print(f"Новый пользователь с ID {new_user_id} добавлен!")
print("Оценки нового пользователя:")
print(new_user_ratings)

train_loader = DataLoader(movielens_train, batch_size, True)
test_loader = DataLoader(movielens_test, batch_size, True)

До добавления нового пользователя:
1 610
0 9741
       userId  movieId  rating   timestamp
0           1     1319     4.0   964981230
1           1      801     5.0   964982400
2           1     2608     4.0   964981775
3           1      981     5.0   964982703
4           1      275     3.0   964982310
...       ...      ...     ...         ...
80664     610     8686     4.0  1479542606
80665     610     9389     3.5  1493848789
80666     610     2970     2.0  1493849562
80667     610     2916     4.5  1493847010
80668     610     5640     5.0  1479545132

[80669 rows x 4 columns]
После добавления нового пользователя:
1 611
0 9741
       userId  movieId  rating   timestamp
0           1      801     5.0   964982400
1           1     2802     4.0   964980694
2           1     2765     5.0   964981909
3           1      136     5.0   964983650
4           1      787     3.0   964982903
...       ...      ...     ...         ...
80684     611     9190     1.0  1732147438
80685     611  

In [7]:
for batch in train_loader:
    for k, v in batch.items():
        print(k, v.shape)
    break

user torch.Size([200, 1])
movie torch.Size([200, 1])
rating torch.Size([200, 1])


In [8]:
# Функции для обучения из прошлой лабы, с учётом юзеров и айтемов
# Функции для обучения из прошлой лабы, с учётом юзеров и айтемов

def train_iteration(model, data_loader, loss_function, optimizer, sheduler):
    model.train()
    train_size = len(data_loader.dataset)
    for idx, batch in enumerate(data_loader):
        batch = {k: v.to(device) for k, v in batch.items()}
        pred = model(batch)
        loss = loss_function(pred, batch['rating'])
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        if idx % 100 == 0:
            loss, current = loss.item(), (idx + 1) * batch_size
            print(f"loss: {loss:>7f}  [{current:>5d}/{train_size:>5d}]")

    scheduler.step(val_loss)

def train(model, train_loader, loss_function, optimizer, scheduler, epochs):
    for t in range(epochs):
        print(f"== Epoch {t + 1} ==")
        model.train() # Переключаем сеть в режим обучения

        total_loss = 0
        correct = 0
        total = 0

        train_size = len(train_loader.dataset)
        for idx, batch in enumerate(train_loader):
            batch = {k: v.to(device) for k, v in batch.items()}   # переносим наши данные на девайс
    
            pred = model(batch)                 # вычисляем предсказание модели
            loss = loss_function(pred, batch['rating'])   # вычисляем потери
            loss.backward()                 # запускаем подсчёт градиентов
    
            optimizer.step()                # делаем шаг градиентного спуска
            optimizer.zero_grad()           # обнуляем градиенты
    
            if idx % 100 == 0:
                
                lossa, current = loss.item(), (idx + 1) * batch_size
                print(f"loss: {lossa:>7f}  [{current:>5d}/{train_size:>5d}]")

            total_loss += loss.item()
        
        # Средние потери за эпоху
        avg_loss = total_loss / len(train_loader)
        scheduler.step(avg_loss)  # Передаем avg_loss в планировщик для корректной работы
                
        test(model, test_loader, loss_function)

#Функция для вывода статистики на тестовом датасете

def test(model, data_loader, loss_function):
    model.eval()
    test_size = len(data_loader.dataset)
    num_batches = len(data_loader)
    loss, correct = 0, 0
    with torch.no_grad():
        for batch in data_loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            pred = model(batch)
            loss += loss_function(pred, batch['rating']).item()
            correct += (pred.argmax(1) == batch['rating']).type(torch.float).sum().item()

    loss /= num_batches
    correct /= test_size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {loss:>8f} \n")

## Матричные разложения

В матричных разложениях используется таблица юзеров-айтемов -- таблица, где по строкам находятся юзеры, по столбцам айтемы, на пересечениях оценка, которую поставил пользователь.

Эта таблица представляется в виде произведения двух матриц, матрицы пользователей и матрицы айтемов

![разложение](images/PQ.drawio.png)

В каждом столбце матрицы пользователей живёт вектор, соответствующий этому пользователю, в матрице айтема, соответственно, вектор айтема. Чтобы получить предсказание оценки, надо их перемножить.

Есть много разных способов находить матричные разложения, поскольку у нас тут pytorch, мы просто возьмём два `Embedding` слоя, перемножим, и скажем что это наша модель, которую обучим градиентным спуском


In [9]:
class MatrixFactorization(nn.Module):
    def __init__(self):
        super().__init__()
        self.user_embeddings  = nn.Embedding(1000,  16)
        self.movie_embeddings = nn.Embedding(10000, 16)

    def forward(self, batch):
        movie_emb = self.user_embeddings(batch['user'])
        user_emb = self.movie_embeddings(batch['movie'])
        return (movie_emb * user_emb).sum(2)


mf_model = MatrixFactorization().to(device)
mf_loss = nn.MSELoss()
mf_optimizer = torch.optim.SGD(mf_model.parameters(), lr=1)

train(10, mf_model, mf_loss, mf_optimizer)

TypeError: train() missing 2 required positional arguments: 'scheduler' and 'epochs'

Фактически, если к этой моделе в сумму добавить общую константу и константу для кадого пользователя и айтема, мы получим Factorization Machine

https://www.ismll.uni-hildesheim.de/pub/pdfs/Rendle2010FM.pdf

А это значит, что помимо эмбедингов с юзерами и айтемами мы можем легко добавить дополнительных параметров! (например тех, что у нас в таблице tags.csv)

## DeepFM

DeepFM это расширение обычной Factorization Machine для использования 

https://arxiv.org/pdf/1703.04247

Идея состоит в том, чтобы расположить рядом с обычной Factorization Machine нейронную сеть, которая будет параллельно существовать с матричным разложением

![DeepFM](images/DeepFM.png)

До DeepFM уже были модели, которые предварительно обучали эмбединги на матричном разложении, а потом использовали их как входные векторы сети, но тут предлагается обучать их сразу совместно

In [ ]:
class DeepFM(nn.Module):
    def __init__(self):
        super().__init__()
        self.user_embeddings  = nn.Embedding(1000,  16)
        self.movie_embeddings = nn.Embedding(10000, 16)

        self.flatten = nn.Flatten()

        self.deep_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(32, 32),
            nn.ReLU(),
            nn.Linear(32, 32),
            nn.ReLU(),
            nn.Linear(32, 32),
            nn.ReLU(),
        )

        self.final_layer = nn.Linear(16*3, 1)

    def forward(self, batch):
        movie_emb = self.flatten(self.user_embeddings(batch['user']))
        user_emb  = self.flatten(self.movie_embeddings(batch['movie']))

        fm = movie_emb * user_emb

        deep = torch.cat([movie_emb, user_emb], 1)
        deep = self.deep_layers(deep)

        v = torch.cat([fm, deep], 1)
        v = self.final_layer(v)
        # делаем сигмоиду на выходе и масштабируем к оценкам от 0 до 5
        return torch.sigmoid(v) * 5


deep_mf_model = DeepFM().to(device)
deep_mf_loss = nn.MSELoss()
deep_mf_optimizer = torch.optim.SGD(deep_mf_model.parameters(), lr=1e-1)

train(5, deep_mf_model, deep_mf_loss, deep_mf_optimizer)

Есть и более прокаченные версии машины факторизации на нейронках, например xDeepFM

https://arxiv.org/pdf/1803.05170

## Задание

Основное задание:
1) Достичь меньше чем 0.8 значения MSELoss на этом датасете (5 баллов)
2) МОЖНО ДЕЛАТЬ ТОЛЬКО ПОСЛЕ ТОГО КАК СДЕЛАНО ПЕРВОЕ ЗАДАНИЕ!  
    Добавить в тренировочный датасет нового пользователя - себя и дать оценки минимум 20 фильмов, обучить модель с учётом этого пользователя и сделать для себя рекомендации. (5 баллов) (пожалуйста, не дописывайте себя в файлик, сделайте пользователя добавление в питоне)

Дополнительные задания:
1) Добавить в модель использование тегов из таблички `tags.csv` (5 дополнительных баллов)
2) Добавить в модель использование дополнительных данных из источников `links.csv` (5 дополнительных баллов)

In [10]:
#улучшенная deepfm

class DeepFM(nn.Module):
    def __init__(self):
        super().__init__()
        
        # Embeddings
        self.user_embeddings = nn.Embedding(1001, 16)
        self.movie_embeddings = nn.Embedding(10020, 16)

        # Deep layers with increased layer sizes, Dropout, and Batch Normalization
        self.deep_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(32, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 32),
            nn.BatchNorm1d(32),
            nn.ReLU(),
        )

        # Final layer
        self.final_layer = nn.Linear(33, 1)

    def forward(self, batch):
        # Получаем эмбеддинги для пользователя и фильма и выравниваем их
        user_emb = self.user_embeddings(batch['user']).view(batch['user'].size(0), -1)  # [batch_size, 16]
        movie_emb = self.movie_embeddings(batch['movie']).view(batch['movie'].size(0), -1)  # [batch_size, 16]
    
        # Factorization Machine (FM) компонент — поэлементное умножение и сумма
        fm = (user_emb * movie_emb).sum(dim=1, keepdim=True)  # [batch_size, 1]
    
        # Deep компонент
        deep = torch.cat([user_emb, movie_emb], dim=1)  # [batch_size, 32]
        deep = self.deep_layers(deep)  # [batch_size, 32]
    
        # Убедимся, что размеры совпадают для конкатенации
        v = torch.cat([fm, deep], dim=1)  # Конкатенируем по dim=1: [batch_size, 33]
        v = self.final_layer(v)  # [batch_size, 1]
    
        # Sigmoid активация для выхода в диапазоне [0, 5]
        return torch.sigmoid(v) * 5



# Initialize model, loss, optimizer, and learning rate scheduler
device = torch.device('cpu' if torch.cuda.is_available() else 'cpu')
deep_mf_model = DeepFM().to(device)
deep_mf_loss = nn.MSELoss()
deep_mf_optimizer = torch.optim.Adam(deep_mf_model.parameters(), lr=1e-3, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(deep_mf_optimizer, mode='min', factor=0.5, patience=2)

train(deep_mf_model, train_loader, deep_mf_loss, deep_mf_optimizer, scheduler, 15)

== Epoch 1 ==
loss: 3.160431  [  200/80689]
loss: 1.446992  [20200/80689]
loss: 1.018417  [40200/80689]
loss: 1.314373  [60200/80689]
loss: 0.986642  [80200/80689]
Test Error: 
 Accuracy: 0.0%, Avg loss: 1.034463 

== Epoch 2 ==
loss: 0.876477  [  200/80689]
loss: 0.954772  [20200/80689]
loss: 0.991723  [40200/80689]
loss: 1.203809  [60200/80689]
loss: 1.054879  [80200/80689]
Test Error: 
 Accuracy: 0.0%, Avg loss: 0.980897 

== Epoch 3 ==
loss: 1.003368  [  200/80689]
loss: 1.178697  [20200/80689]
loss: 0.871857  [40200/80689]
loss: 1.098128  [60200/80689]
loss: 1.004603  [80200/80689]
Test Error: 
 Accuracy: 0.0%, Avg loss: 0.952163 

== Epoch 4 ==
loss: 0.856993  [  200/80689]
loss: 0.820415  [20200/80689]
loss: 0.883067  [40200/80689]
loss: 0.881918  [60200/80689]
loss: 0.945334  [80200/80689]
Test Error: 
 Accuracy: 0.0%, Avg loss: 0.941201 

== Epoch 5 ==
loss: 0.902015  [  200/80689]
loss: 0.961218  [20200/80689]
loss: 1.054953  [40200/80689]
loss: 0.887488  [60200/80689]
loss: 

In [11]:
def get_movie_recommendations(model, user_id, n_recommendations=10, device='cuda'):
    """
    Получить рекомендации для конкретного пользователя.

    :param model: Обученная модель
    :param user_id: ID пользователя
    :param n_recommendations: Количество рекомендаций
    :param device: Устройство ('cuda' или 'cpu')
    :return: Список фильмов с рекомендованными рейтингами
    """
    model.eval()  # Переводим модель в режим оценки
    with torch.no_grad():
        # Получаем эмбеддинги для пользователя
        user_tensor = torch.LongTensor([user_id]).to(device)
        
        # Генерация всех возможных movieId
        all_movie_ids = torch.LongTensor(np.arange(10000)).to(device)  # 10000 - количество фильмов
        
        # Строим батч для вычисления предсказаний
        batch = {'user': user_tensor.repeat(len(all_movie_ids), 1), 'movie': all_movie_ids.view(-1, 1)}
        
        # Прогоняем через модель для получения рейтингов
        predictions = model(batch).cpu().numpy()
        
        # Выбираем топ-N фильмов по предсказанным рейтингам
        top_n_indices = np.argsort(predictions.flatten())[-n_recommendations:][::-1].copy()
        
        # Создаем массив идентификаторов фильмов на основе полученных индексов
        top_n_movie_ids = all_movie_ids[top_n_indices].numpy()

        return top_n_movie_ids, predictions.flatten()[top_n_indices]

# Пример использования:
user_id = 610  # ID пользователя, для которого генерируем рекомендации
n_recommendations = 10  # Количество рекомендаций

# Получаем рекомендации
recommended_movie_ids, predicted_ratings = get_movie_recommendations(deep_mf_model, user_id, n_recommendations, device)

# Выводим рекомендации
print(f"Топ-{n_recommendations} рекомендаций для пользователя {user_id}:")
for movie_id, rating in zip(recommended_movie_ids, predicted_ratings):
    print(f"Movie ID: {movie_id}, Predicted Rating: {rating:.2f}")

Топ-10 рекомендаций для пользователя 610:
Movie ID: 277, Predicted Rating: 4.54
Movie ID: 4909, Predicted Rating: 4.52
Movie ID: 694, Predicted Rating: 4.51
Movie ID: 922, Predicted Rating: 4.49
Movie ID: 46, Predicted Rating: 4.47
Movie ID: 461, Predicted Rating: 4.47
Movie ID: 863, Predicted Rating: 4.46
Movie ID: 899, Predicted Rating: 4.45
Movie ID: 914, Predicted Rating: 4.43
Movie ID: 982, Predicted Rating: 4.43


## Бонусное изменение моделей

In [60]:
class newDeepFM(nn.Module):
    def __init__(self, tag_embedding_size=8, max_tags=5):
        super().__init__()

        # Embeddings
        self.user_embeddings = nn.Embedding(1001, 16)  # 1001 пользователей, 16 размерность
        self.movie_embeddings = nn.Embedding(10020, 16)  # 10020 фильмов, 16 размерность
        self.tag_embeddings = nn.Embedding(1000, tag_embedding_size, padding_idx=0)  # 1000 тегов, 8 размерность

        # Deep layers
        input_size = 16 + 16 + tag_embedding_size + 2  # Усредняем теги, поэтому только tag_embedding_size
        self.deep_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(input_size, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 32),
            nn.BatchNorm1d(32),
            nn.ReLU(),
        )

        # Final layer
        self.final_layer = nn.Linear(33, 1)

    def forward(self, batch):
        # Embeddings
        user_emb = self.user_embeddings(batch['user']).view(batch['user'].size(0), -1)  # [batch_size, 16]
        movie_emb = self.movie_embeddings(batch['movie']).view(batch['movie'].size(0), -1)  # [batch_size, 16]
    
        # Тег-эмбеддинги: обработка тегов
        tags = batch['tags'].clamp(0, self.tag_embeddings.num_embeddings - 1)  # Ограничиваем индексы
        tag_emb = self.tag_embeddings(tags)  # [batch_size, max_tags, tag_embedding_size]
        tag_emb = tag_emb.mean(dim=1)  # Усреднение по тегам, [batch_size, tag_embedding_size]
    
        # Дополнительные фичи
        link_features = batch['link_features']  # [batch_size, 2]
    
        # FM Component
        fm = (user_emb * movie_emb).sum(dim=1, keepdim=True)  # [batch_size, 1]
    
        # Deep Component
        deep = torch.cat([user_emb, movie_emb, tag_emb, link_features], dim=1)  # [batch_size, input_size]
        deep = self.deep_layers(deep)  # [batch_size, 32]
    
        # Combine FM and Deep components
        v = torch.cat([fm, deep], dim=1)  # [batch_size, 33]
        v = self.final_layer(v)  # [batch_size, 1]
    
        # Output scaled to [0, 5]
        return torch.sigmoid(v) * 5



In [61]:
class MovielensDataset(Dataset):
    def __init__(self, source, train=True, seed=1, max_tags=5):
        ratings = pd.read_csv(rf"{source}\ratings.csv")
        self.movies = pd.read_csv(rf"{source}\movies.csv")
        self.tags = pd.read_csv(rf"{source}\tags.csv")
        self.links = pd.read_csv(rf"{source}\links.csv")

        # Преобразуем ID фильмов в индексы
        x = self.movies.loc[:, ['movieId']]
        x['movieId'], x.index = x.index, x['movieId'].values
        ratings['movieId'] = ratings['movieId'].map(x.to_dict()['movieId'])
        self.links['movieId'] = self.links['movieId'].map(x.to_dict()['movieId'])
        self.tags['movieId'] = self.tags['movieId'].map(x.to_dict()['movieId'])

        # Словарь тегов для каждого фильма
        self.movie_to_tags = self._preprocess_tags(max_tags)

        # Нормализуем данные links
        self.link_features = self._preprocess_links()

        # Разделение на обучающую и тестовую выборки
        train_data = ratings.sample(frac=0.8, random_state=seed)
        test_data = ratings.drop(train_data.index)
        self.ratings = train_data if train else test_data

    def __len__(self):
        return len(self.ratings)

    def __getitem__(self, idx):
        sample = self.ratings.iloc[idx]

        # Теги
        tags = self.movie_to_tags.get(sample['movieId'], [0] * 5)  # Паддинг нулями

        # Фичи из links.csv
        link_features = self.link_features[sample['movieId']]

        return {
            "user": torch.LongTensor([sample['userId']]),
            "movie": torch.LongTensor([sample['movieId']]),
            "rating": torch.FloatTensor([sample['rating']]),
            "tags": torch.LongTensor(tags),  # Теги как индексы
            "link_features": torch.FloatTensor(link_features)  # Нормализованные фичи
        }

    def _preprocess_tags(self, max_tags):
        """Преобразование тегов в индексы."""
        unique_tags = list(set(self.tags['tag'].values))
        tag_to_idx = {tag: idx + 1 for idx, tag in enumerate(unique_tags)}  # 0 для паддинга

        # Формируем словарь {movieId: [tag_idx, ...]}
        movie_to_tags = defaultdict(list)
        for _, row in self.tags.iterrows():
            movie_to_tags[row['movieId']].append(tag_to_idx[row['tag']])

        # Ограничиваем количество тегов
        for movie_id, tag_list in movie_to_tags.items():
            movie_to_tags[movie_id] = tag_list[:max_tags] + [0] * (max_tags - len(tag_list))

        return movie_to_tags

    def _preprocess_links(self):
        """Нормализация данных из links.csv."""
        features = self.links[['imdbId', 'tmdbId']].fillna(0).values
        max_vals = features.max(axis=0)
        features = features / max_vals  # Нормализация в диапазоне [0, 1]

        # Создаем словарь {movieId: [normalized_features]}
        link_features = {row['movieId']: features[i] for i, row in self.links.iterrows()}
        return link_features

    def add_new_user(self, seed=1, num_movies=20):
        """Добавление нового пользователя с его оценками."""
        new_user_id = self.ratings['userId'].max() + 1

        np.random.seed(seed)
        sampled_movie_ids = self.ratings['movieId'].unique()[:20]
        sampled_ratings = np.round(np.random.uniform(1.0, 5.0, num_movies) * 2) / 2

        new_user_ratings = pd.DataFrame({
            'userId': new_user_id,
            'movieId': sampled_movie_ids,
            'rating': sampled_ratings,
            'timestamp': int(pd.Timestamp.now().timestamp())
        })

        self.ratings = pd.concat([self.ratings, new_user_ratings], ignore_index=False)
        return new_user_id


In [62]:
batch_size = 200

sataset_source = r'.\ml-latest-small'

movielens_train = MovielensDataset(sataset_source, train=True)
movielens_test  = MovielensDataset(sataset_source, train=False)

train_loader = DataLoader(movielens_train, batch_size, True)
test_loader = DataLoader(movielens_test, batch_size, True)

In [63]:
for idx, batch in enumerate(train_loader):
    batch = {k: v.to(device) for k, v in batch.items()}
    batch['tags'] = batch['tags'].clamp(0, 999)
    print("Tags in batch:", batch['tags'])
    print("Max tag index:", batch['tags'].max().item())
    print("Min tag index:", batch['tags'].min().item())
    break

Tags in batch: tensor([[999,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0],
        [263,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0],
        [443, 999,   0,   0,   0],
        [  0,   0,   0,   0,   0],
        [440, 875, 999, 999,   0],
        [998, 861,   0,   0,   0],
        [  0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0],
        [805, 361, 999, 389, 243],
        [758, 758, 217,   0,   0],
        [611, 999, 999,   0,   0],
        [  0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0],
        [235, 530, 598,  34,   0],
        [  0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0],
        [999, 190, 999, 918, 886],
        [999,  91, 999, 999, 999],
        [  0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0],
     

In [64]:
device = torch.device('cpu' if torch.cuda.is_available() else 'cpu')
deep_mf_model = newDeepFM().to(device)
deep_mf_loss = nn.MSELoss()
deep_mf_optimizer = torch.optim.Adam(deep_mf_model.parameters(), lr=1e-3, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(deep_mf_optimizer, mode='min', factor=0.5, patience=2)

train(deep_mf_model, train_loader, deep_mf_loss, deep_mf_optimizer, scheduler, 15)

== Epoch 1 ==
loss: 4.171557  [  200/80669]
loss: 1.118802  [20200/80669]
loss: 0.999063  [40200/80669]
loss: 1.134953  [60200/80669]
loss: 1.070572  [80200/80669]
Test Error: 
 Accuracy: 0.0%, Avg loss: 1.021810 

== Epoch 2 ==
loss: 0.973078  [  200/80669]
loss: 1.152095  [20200/80669]
loss: 0.954809  [40200/80669]
loss: 1.006898  [60200/80669]
loss: 0.970310  [80200/80669]
Test Error: 
 Accuracy: 0.0%, Avg loss: 0.969078 

== Epoch 3 ==
loss: 0.918209  [  200/80669]
loss: 1.140400  [20200/80669]
loss: 0.959102  [40200/80669]
loss: 1.033707  [60200/80669]
loss: 0.819741  [80200/80669]
Test Error: 
 Accuracy: 0.0%, Avg loss: 0.943036 

== Epoch 4 ==
loss: 0.915161  [  200/80669]
loss: 0.916775  [20200/80669]
loss: 0.803124  [40200/80669]
loss: 1.080952  [60200/80669]
loss: 0.784749  [80200/80669]
Test Error: 
 Accuracy: 0.0%, Avg loss: 0.908509 

== Epoch 5 ==
loss: 1.240274  [  200/80669]
loss: 0.892292  [20200/80669]
loss: 0.804601  [40200/80669]
loss: 0.883409  [60200/80669]
loss: 